# A2: Monte Carlo


## Monte Carlo Part A
>1. Explain clearly why V_pi is not useful in the MC development above?

Because state values V_pi alone simply leads us to the best combination of reward and next state S' following the policy pi. However, without the the transition dynamics p, we don't know what action can take us to state S'. 

>2. The MC algorithm so far (ref: p 99), requires an infinite number of episodes for Eval to converge on Q_pi_k (step k). We can modify this algorithm to the practical variant where Eval is truncated (c.f., DynProg GPI). In this case: 
>* a. Will we obtain Q_pi_k from eval?

No. Since evaluation is truncated, we can only obtain an approximate value.
>* b. If not why are we able to truncate Eval? Explain clearly.

Because by implementing truncated evaluation and improvement repeatedly, the value function is altered to more closely approximate to true value function for the current policy, and the policy is improved concerning the current value function. These two kinds of changes together cause both the value function and the policy to converge to the optimality.


>* c. Assuming ES (i.e., thorough sampling of the S x A space), and the above truncated Eval_trunc, is it possible to converge on a sub-optimal policy pi_c? Is this a stable fixed point of the GPI for MC? Explain clearly.

It cannot converge to any sub-optimal policy. Because if it converges on a sub-optimal policy pi_c, then throughout policy evaluation repeatedly, the value function will converge to the real value function under policy pi_c. Then policy improvement will guarantee a new policy better than policy pi_c. So it will eventually lead to the optimal policy.
>3. Explain how you can synthesize a stochastic policy given what you know so far (you don't need to read ahead).

Start with a policy with non-zero values for each state-action pair so that all state-action pairs will be visited in a finite number of episodes.






##Monte Carlo Part B-1: Stochastic Policies

##Monte Carlo Part B-2: Off Policy Methods
>Code the algorithm for MC Control (Off Policy) and apply this to the Cart Pole problem. You must discretize the environmental feedback (S) in order to solve this problem properly.

In [1]:
import gym
import numpy as np
import matplotlib.pyplot as plt
import collections

In [4]:
env = gym.make("CartPole-v0")
env.reset()


for i in range(50000):
  action = env.action_space.sample()
  print("step i",i,"action=",action)
  obs, reward, done, info = env.step(action)
  print("obs=",obs,"reward=",reward,"done=",done,"info=",info)

  if done:
    break
    
env.close()
print("Iterations that were run:",i)

step i 0 action= 0
obs= [ 0.03098964 -0.21151883  0.03340614  0.27969223] reward= 1.0 done= False info= {}
step i 1 action= 0
obs= [ 0.02675926 -0.40710101  0.03899998  0.58272139] reward= 1.0 done= False info= {}
step i 2 action= 0
obs= [ 0.01861724 -0.60274703  0.05065441  0.88743035] reward= 1.0 done= False info= {}
step i 3 action= 1
obs= [ 0.0065623  -0.40834786  0.06840302  0.61109165] reward= 1.0 done= False info= {}
step i 4 action= 0
obs= [-0.00160466 -0.60435585  0.08062485  0.92451087] reward= 1.0 done= False info= {}
step i 5 action= 0
obs= [-0.01369178 -0.80046884  0.09911507  1.2414028 ] reward= 1.0 done= False info= {}
step i 6 action= 1
obs= [-0.02970115 -0.60674907  0.12394312  0.98134142] reward= 1.0 done= False info= {}
step i 7 action= 1
obs= [-0.04183613 -0.41348647  0.14356995  0.73001739] reward= 1.0 done= False info= {}
step i 8 action= 1
obs= [-0.05010586 -0.22060986  0.1581703   0.48574379] reward= 1.0 done= False info= {}
step i 9 action= 1
obs= [-0.05451806 

In [5]:
#discretize the environmental feedback (S)
'''
  Type: Box(4)
  Num     Observation               Min                     Max
  0       Cart Position             -4.8                    4.8
  1       Cart Velocity             -Inf                    Inf
  2       Pole Angle                -0.418 rad (-24 deg)    0.418 rad (24 deg)
  3       Pole Angular Velocity     -Inf                    Inf
'''

def discretize_state(obs):
	# env.observation_space.high
	# [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]
	# env.observation_space.low
	# [-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38]
  discrete = [np.digitize(obs[i], bins) for i, bins in enumerate([
    np.linspace(-4.8, 4.8, 9),
    np.linspace(-4, 4, 7),
    np.linspace(-0.418, 0.418, 9),
    np.linspace(-4, 4, 7),
    ])]
  return ((obs > 0) * 8 **np.arange(len(obs))).sum()

In [3]:
#policy initialization
def init_policy(epsilon,S,A):
  pi_init = np.random.random([S,A])
  out = np.zeros_like(pi_init, dtype = np.float)
  idx = pi_init.argmax(axis=1)
  out[np.arange(S), idx] = 1
  pi = out*(1 - epsilon) + epsilon / A
  return pi

In [10]:
#generate episode using b
def generate_episode(env, policy, actions):
  episode = []
  obs = env.reset()
  i = 0
  while True:
    i += 1
    state = discretize_state(obs)
    action = np.random.choice(actions, p=policy[state])
    obs, reward, done, info = env.step(action)
    episode.append((state, action, reward))
    if done:
      break
  return episode,i


In [12]:
#MC Control (Off Policy)
def Monte_Carlo_off_policy(env,gamma,epsilon):
  S = 8**len(obs)
  A = env.action_space.n
  #initialize
  states = np.arange(S)
  actions = np.arange(A)
  Q = np.random.random([S,A])
  C = np.zeros([S,A])
  #initialize pi
  pi = np.argmax(Q, axis = 1)


  for episode in range(1000):
    b = init_policy(epsilon,S,A)
    #generate episodes
    episode, i = generate_episode(env,b,actions)
    #state_actions = [(s, a) for (s,a,r) in episode]
    
    G = 0
    W = 1
    for t in range(i-1,-1,-1):
      state, action, reward = episode[t]
      G = gamma * G + reward
      C[state,action] += W
      Q[state,action] += W/C[state,action]*(G-Q[state,action])
      pi[state] = np.argmax(Q[state]) 
      if action != pi[state]:
        break
      W = W / b[state,action]

  return Q, pi

In [15]:
Q,pi = Monte_Carlo_off_policy(env,0.99,0.01)

In [24]:
A = env.action_space.n
# Test
done = False
obs = env.reset()
for i in range(50000):
    state = discretize_state(obs)
    action = pi[state]

    print("step i",i,"action=",action)
    obs, reward, done, info = env.step(action)
    print("obs=",obs,"reward=",reward,"done=",done,"info=",info)

    if done:
        break

step i 0 action= 1
obs= [-0.0293961   0.23138337 -0.02315883 -0.26607751] reward= 1.0 done= False info= {}
step i 1 action= 0
obs= [-0.02476843  0.03659948 -0.02848038  0.0192119 ] reward= 1.0 done= False info= {}
step i 2 action= 1
obs= [-0.02403644  0.23211805 -0.02809614 -0.28231912] reward= 1.0 done= False info= {}
step i 3 action= 0
obs= [-0.01939408  0.03740788 -0.03374252  0.00137168] reward= 1.0 done= False info= {}
step i 4 action= 1
obs= [-0.01864592  0.23299709 -0.03371509 -0.30176356] reward= 1.0 done= False info= {}
step i 5 action= 0
obs= [-0.01398598  0.03837149 -0.03975036 -0.01990141] reward= 1.0 done= False info= {}
step i 6 action= 0
obs= [-0.01321855 -0.15615852 -0.04014839  0.25997957] reward= 1.0 done= False info= {}
step i 7 action= 1
obs= [-0.01634172  0.03951288 -0.0349488  -0.04509147] reward= 1.0 done= False info= {}
step i 8 action= 0
obs= [-0.01555147 -0.15509094 -0.03585063  0.23636316] reward= 1.0 done= False info= {}
step i 9 action= 1
obs= [-0.01865328 